<a href="https://colab.research.google.com/github/jssaini7815/Basic-Python/blob/main/HandWrittenDigit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step 2
#CNN model builder

In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# Input image dimensions
img_rows, img_cols = 28, 28

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Rescale the image values to [0, 1]
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Set the CNN Architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Comple the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Save the model weights for future reference
model.save('mnist_cnn_model.h5')

# Evaluate the model using Accuracy and Loss
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/12
469/469 [==============================] - 7s 15ms/step - loss: 2.2829 - accuracy: 0.1378 - val_loss: 2.2144 - val_accuracy: 0.3391
Epoch 2/12
469/469 [==============================] - 7s 14ms/step - loss: 2.2075 - accuracy: 0.2609 - val_loss: 2.1121 - val_accuracy: 0.5814
Epoch 3/12
469/469 [==============================] - 7s 14ms/step - loss: 2.1068 - accuracy: 0.3935 - val_loss: 1.9654 - val_accuracy: 0.6980
Epoch 4/12
469/469 [==============================] - 7s 14ms/step - loss: 1.9627 - accuracy: 0.4929 - val_loss: 1.7666 - val_accuracy: 0.7547
Epoch 5/12
469/469 [==============================] - 7s 14ms/step - loss: 1.7796 - accuracy: 0.5588 - val_loss: 1.5363 - val_accuracy: 0.7806
Epoch 6/12
469/469 [==============================] - 7s 14ms/step - loss: 1.5858 - accuracy: 0.5946 - val_loss: 1.3055 - val_accuracy: 0.7980
Epoch 7/12
469/469 [==============================] - 7s 14ms/step - loss: 1.4078 - accuracy: 0.6284 - val_loss: 1.1059 - val_accuracy: 0.8119

In [ ]:
#Step 3
#Digits recognition

In [5]:
from keras.models import load_model
from collections import deque
import numpy as np
import cv2

# Load the models built in the previous steps
mlp_model = load_model('mnist_mlp_model.h5')
cnn_model = load_model('mnist_cnn_model.h5')

# Define the upper and lower boundaries for a color to be considered "Blue"
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Define Black Board
blackboard = np.zeros((480,640,3), dtype=np.uint8)
digit = np.zeros((200, 200, 3), dtype=np.uint8)

# Setup deques to store digits drawn on screen
points = deque(maxlen=512)

# Define answer variables
ans1 = ' '
ans2 = ' '

index = 0
# Load the video
camera = cv2.VideoCapture(0)

# Keep looping
while True:
    # Grab the current paintWindow
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    # Find contours (bottle cap in my case) in the image
    (_, cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
    	cv2.CHAIN_APPROX_SIMPLE)
    center = None

    # Check to see if any contours were found
    if len(cnts) > 0:
    	# Sort the contours and find the largest one -- we
    	# will assume this contour correspondes to the area of the bottle cap
        cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # Get the radius of the enclosing circle around the found contour
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        # Draw the circle around the contour
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        # Get the moments to calculate the center of the contour (in this case Circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

        points.appendleft(center)

    elif len(cnts) == 0:
        if len(points) != 0:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
            if len(blackboard_cnts) >= 1:
                cnt = sorted(blackboard_cnts, key = cv2.contourArea, reverse = True)[0]

                if cv2.contourArea(cnt) > 1000:
                    x, y, w, h = cv2.boundingRect(cnt)
                    digit = blackboard_gray[y-10:y + h + 10, x-10:x + w + 10]
                    newImage = cv2.resize(digit, (28, 28))
                    newImage = np.array(newImage)
                    newImage = newImage.astype('float32')/255

                    ans1 = mlp_model.predict(newImage.reshape(1, 28, 28))[0]
                    ans1 = np.argmax(ans1)
                    ans2 = cnn_model.predict(newImage.reshape(1,28,28,1))[0]
                    ans2 = np.argmax(ans2)

            # Empty the points deque and the blackboard
            points = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)

    # Connect the points with a line
    for i in range(1, len(points)):
            if points[i - 1] is None or points[i] is None:
                    continue
            cv2.line(frame, points[i - 1], points[i], (0, 0, 0), 2)
            cv2.line(blackboard, points[i - 1], points[i], (255, 255, 255), 8)

    # Put the result on the screen
    cv2.putText(frame, "Multilayer Perceptron : " + str(ans1), (10, 410), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(255, 255, 255), 2)
    cv2.putText(frame, "Convolution Neural Network:  " + str(ans2), (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Show the frame
    cv2.imshow("Digits Recognition Real Time", frame)

    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()


error: ignored

In [ ]:
#Step 1
#mlp_model_builder

In [2]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint

# Use Keras to import pre-shuffled MNIST database
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("The MNIST database has a training set of %d examples." % len(X_train))
print("The MNIST database has a test set of %d examples." % len(X_test))


# Rescale the Images by Dividing Every Pixel in Every Image by 255
# Rescale [0,255] --> [0,1]

X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255


# Encode Categorical Integer Labels Using a One-Hot Scheme

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Define the Model Architecture

model = Sequential()
model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# Summarize the model
# model.summary()


# Compile the Model

model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])


# Calculate the Classification Accuracy on the Test Set (Before Training)

score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

print('Before Training - Test accuracy: %.4f%%' % accuracy)


# Train the Model

hist = model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_split=0.2, verbose=1, shuffle=True)


# Load the Model with the Best Classification Accuracy on the Validation Set

model.save('mnist_mlp_model.h5')

# Calculate the Classification Accuracy on the Test Set

score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

print('After Training - Test accuracy: %.4f%%' % accuracy)


The MNIST database has a training set of 60000 examples.
The MNIST database has a test set of 10000 examples.
Before Training - Test accuracy: 8.6900%
Epoch 1/10
375/375 [==============================] - 3s 5ms/step - loss: 0.2749 - accuracy: 0.9154 - val_loss: 0.1190 - val_accuracy: 0.9648
Epoch 2/10
375/375 [==============================] - 2s 5ms/step - loss: 0.1105 - accuracy: 0.9662 - val_loss: 0.0919 - val_accuracy: 0.9741
Epoch 3/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0805 - accuracy: 0.9752 - val_loss: 0.0892 - val_accuracy: 0.9749
Epoch 4/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0626 - accuracy: 0.9807 - val_loss: 0.1011 - val_accuracy: 0.9743
Epoch 5/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0512 - accuracy: 0.9845 - val_loss: 0.0949 - val_accuracy: 0.9773
Epoch 6/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0445 - accuracy: 0.9864 - val_loss: 0.0915 - val_accuracy: 0.97